In [1]:
# @title Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Information Retrieval Benchmark Data Download

This notebook helps to download data from the [BEIR-cellar](https://github.com/beir-cellar/beir/wiki/Datasets-available)[1] to GCS so that it can be used in benchmarking different information retrieval strategies. The general format of this data is that the queries and documents are jsonl files where each line has the keys `_id`, `title` and `text` (See https://github.com/beir-cellar/beir for more information). A subset of the queries and the corpus should be included in a train.tsv file that has the columns query-id, corpus-id and score. The query-id is a string that matches the _id key from the queries jsonl file, the corpus-id is the same but for the corpus and any score with a value greater than 0 indicates that the document is related to the query. Larger numbers indicate a greater level of relevance. A test.tsv file of the same format should also be provided to assess the performance of the customized model. For a concrete example, data from the [CQADupStack English](http://nlp.cis.unimelb.edu.au/resources/cqadupstack/)[2] dataset is retrieved in this colab.

The downloaded data can then be used to test different Information Retrieval methods such as semantic search using customized embeddings.

This notebook requires that you have a GCP project and permissions to write objects to GCS. To train the custom embedding model the service agent `service-<project_number>@gcp-sa-aiplatform.iam.gserviceaccount.com` must be able to read and write objects to the GCS buckets. See the documentation for more information about the [Vertex service agents](https://cloud.google.com/vertex-ai/docs/general/access-control#service-agents) and [Cloud Storage permissions](https://cloud.google.com/vertex-ai/docs/general/access-control#storage-roles).

####References
[1]: Thakur, Nandan, et al. "BEIR: A heterogeneous benchmark for zero-shot evaluation of information retrieval models." arXiv preprint arXiv:2104.08663 (2021).

[2]: Hoogeveen, Doris, Karin M. Verspoor, and Timothy Baldwin. "CQADupStack: A benchmark data set for community question-answering research." Proceedings of the 20th Australasian document computing symposium. 2015.


In [2]:
# Install dependencies.
! pip install -U "gcsfs==2023.4.0" "tqdm==4.65.0" -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
huggingface-hub 0.18.0 requires fsspec>=2023.5.0, but you have fsspec 2023.4.0 which is incompatible.
llama-index 0.8.58 requires fsspec>=2023.5.0, but you have fsspec 2023.4.0 which is incompatible.
llama-index 0.8.58 requires urllib3<2, but you have urllib3 2.0.4 which is incompatible.
llava 1.1.3 requires scikit-learn==1.2.2, but you have scikit-learn 1.3.1 which is incompatible.


**Attention**: you may need to restart runtime so that the right package is installed.

In [6]:
# Import dependencies.
import base64
from collections.abc import Sequence
import datetime
import glob
import hashlib
import io
import os
import shutil
import tempfile
from typing import Optional, Union
import zipfile

from google.cloud import storage
import pandas as pd
import requests
import tensorflow as tf
import tqdm

2023-11-21 06:59:11.606461: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-21 06:59:11.667336: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-21 06:59:11.667376: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-21 06:59:11.668780: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-21 06:59:11.678420: I tensorflow/core/platform/cpu_feature_guar

### Configure all the things here 

In [13]:
#@markdown Authenticate and define constants to be used across the demo. { display-mode: "form" }
# Authenticate with GCP to run Vertex jobs and save data.


# This will bringup a pop-up window.
# from google.colab import auth as google_auth
# google_auth.authenticate_user()

#@markdown Set your project name and the bucket name. The bucket must either exist or you have permissions to create a bucket. Either way you must have object writer permissions to the bucket.
# PROJECT_ID = '' # @param {type:"string"}
BUCKET_NAME = 'my-project-0004-bucket' # @param {type:"string"}
#@markdown These parameters help specify the download data and locations and can be left as the defaults.
INPUT_DATA_PREFIX = 'download/beir' # @param {type:"string"}
LOCATION = 'us-central1' # @param {type:"string"}
# See https://github.com/beir-cellar/beir#beers-available-datasets for datasets.
BEIR_DATASET_NAME = 'cqadupstack/english' # @param {type:"string"}
URL_DATASET_NAME = BEIR_DATASET_NAME.split('/')[0]


REGION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "text-bison@001" # @param {type:"string"}
IMAGE_MODEL_NAME = "imagegeneration" # @param {type:"string"}

#gs://my-project-0004-bucket/matching_engine/

VERTEX_API_PROJECT = PROJECT_ID = "my-project-0004-346516" #'your-project' #@param {"type": "string"}
VERTEX_API_LOCATION =REGION= 'us-central1' #@param {"type": "string"}


OUTPUT_SUBFOLDER = "output"
TEMP_DIR = tempfile.mkdtemp()

# See https://github.com/beir-cellar/beir
DATA_URL = f"https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{URL_DATASET_NAME}.zip"


MAX_COL_WIDTH = 100
pd.set_option('display.max_colwidth', MAX_COL_WIDTH)
# Create a global GCS Client.
STORAGE_CLIENT = storage.Client(project=PROJECT_ID)


In [14]:
# @markdown Define helper functions for loading the data. { display-mode: "form" }


def create_hash_filename(
    input_str: str, ext: str = '.zip', max_len: int = 48
) -> str:
  hash_filename = hashlib.sha256(input_str.encode('utf8')).hexdigest()
  return f'{hash_filename[:max_len]}{ext}'


# Define helper functions
def _upload_to_gcs(
    base_path: str, bucket: storage.Bucket, gcs_path: str
) -> Sequence[str]:
  output_paths = []
  for local_path in glob.glob(f'{base_path}/**'):
    file_name = os.path.basename(local_path)
    if os.path.isdir(local_path):
      output_paths.extend(
          _upload_to_gcs(local_path, bucket, f'{gcs_path}/{file_name}')
      )
    else:
      remote_path = f'{gcs_path}/{file_name}'
      full_gcs_path = f'gs://{bucket.name}/{remote_path}'
      print(f'Uploading {local_path} to {full_gcs_path}')
      tf.io.gfile.copy(local_path, full_gcs_path)
      output_paths.append(full_gcs_path)

  return output_paths


_DEFAULT_CHUNK_SIZE = 1024 * 1024 * 10


def download_file(
    download_url: str,
    bucket_name: str,
    base_gcs_prefix: str,
    ignore_cache: bool = False,
    chunk_size: int = _DEFAULT_CHUNK_SIZE,
) -> str:
  session = requests.Session()
  response = session.get(download_url, stream=True)
  file_name = create_hash_filename(download_url)
  zip_file_path = os.path.join(TEMP_DIR, file_name)
  if ignore_cache or (not os.path.isfile(zip_file_path)):
    os.makedirs(os.path.dirname(zip_file_path), exist_ok=True)
    content_length = int(response.headers.get('content-length', 0))
    with (
        open(zip_file_path, 'wb') as f,
        tqdm.tqdm(
            total=content_length, leave=False, unit='B', unit_scale=True
        ) as pbar,
    ):
      for data in response.iter_content(chunk_size=chunk_size):
        f.write(data)
        pbar.update(chunk_size)
    print(f'Finished downloading {download_url} to {zip_file_path}')
  else:
    print(f'Using cached version of {zip_file_path}')

  return zip_file_path


def read_beir_tsv(file_path: str):
  return pd.read_csv(
      file_path,
      sep='\t',
      dtype={'query-id': str, 'corpus-id': str, 'score': int},
  )


def write_beir_test(file_path: str, df: pd.DataFrame):
  df.to_csv(file_path, sep='\t', index=False)


def generate_train_test_split(
    input_path: str, train_split: float = 0.7
) -> tuple[pd.DataFrame, pd.DataFrame]:
  full_dataset = read_beir_tsv(input_path)
  full_dataset = full_dataset.sort_values('query-id', ignore_index=True)
  unique_queries = full_dataset['query-id'].unique()
  last_query = unique_queries[int(len(unique_queries) * train_split)]
  query_loc = full_dataset['query-id'].searchsorted(last_query, side='right')

  train_data = full_dataset.iloc[:query_loc].reset_index(drop=True)
  test_data = full_dataset.iloc[query_loc:].reset_index(drop=True)
  return train_data, test_data


def _normalize_gcs_path(gcs_path: str) -> str:
  return gcs_path if gcs_path.startswith('gs://') else f'gs://{gcs_path}'


ALL_FILES = (
    'corpus.jsonl',
    'queries.jsonl',
    'qrels/train.tsv',
    'qrels/test.tsv',
)


def files_exist_on_gcs(
    bucket_name: str,
    base_gcs_prefix: str,
    dataset_prefix: Optional[str] = None,
    expected_files: Sequence[str] = ALL_FILES,
) -> list[str]:
  output_paths = []
  for f_name in expected_files:
    if dataset_prefix:
      full_path = os.path.join(
          bucket_name, base_gcs_prefix, dataset_prefix, f_name
      )
    else:
      full_path = os.path.join(bucket_name, base_gcs_prefix, f_name)

    full_path = _normalize_gcs_path(full_path)
    if not tf.io.gfile.exists(full_path):
      print(f'{full_path} does not exist')
      return []
    else:
      output_paths.append(full_path)

  return output_paths


def _get_gcs_bucket(bucket_name: str) -> storage.Bucket:
  bucket_name = bucket_name.split('gs://')[-1]

  if not bucket_name in {b.name for b in STORAGE_CLIENT.list_buckets()}:
    bucket = STORAGE_CLIENT.create_bucket(bucket_name, location=LOCATION)
  else:
    bucket = STORAGE_CLIENT.bucket(bucket_name)
  return bucket


def download_to_gcs(
    download_url: str,
    bucket_name: str,
    base_gcs_prefix: str,
    ignore_cache: bool = False,
) -> dict[str, str]:
  if (
      uploaded_paths := files_exist_on_gcs(
          bucket_name, base_gcs_prefix, BEIR_DATASET_NAME
      )
  ) and not ignore_cache:
    print(f'Using existing files on GCS: {uploaded_paths}')
    return uploaded_paths

  file_path = download_file(
      download_url=download_url,
      bucket_name=bucket_name,
      base_gcs_prefix=base_gcs_prefix,
      ignore_cache=ignore_cache,
  )
  output_dir = tempfile.mkdtemp()

  with zipfile.ZipFile(file_path) as z:
    for f_name in z.namelist():
      if f_name.startswith(BEIR_DATASET_NAME):
        print(f'Extracting {f_name} from {file_path}')
        z.extract(f_name, output_dir)

  # Since CQADupStack English only has a test.tsv file split it into a train and
  # test set.
  qrels_dir = os.path.join(output_dir, BEIR_DATASET_NAME, 'qrels')
  train_path = os.path.join(qrels_dir, 'train.tsv')
  test_path = os.path.join(qrels_dir, 'test.tsv')
  if not (os.path.exists(train_path) or os.path.exists(test_path)):
    raise ValueError(
        f'Either test or train or both files must be present in {qrels_dir}'
    )

  if not os.path.exists(train_path):
    train_data, test_data = generate_train_test_split(test_path)
    write_beir_test(train_path, train_data)
    write_beir_test(test_path, test_data)
  elif not os.path.exists(test_path):
    train_data, test_data = generate_train_test_split(train_path)
    write_beir_test(train_path, train_data)
    write_beir_test(test_path, test_data)

  bucket = _get_gcs_bucket(bucket_name)
  uploaded_paths = _upload_to_gcs(output_dir, bucket, base_gcs_prefix)
  # Clean up after ourselves.
  shutil.rmtree(output_dir)
  return uploaded_paths


def get_path_by_type(all_file_paths: Sequence[str]) -> dict[str, str]:
  path_by_type = {}
  for f in all_file_paths:
    if f.endswith('queries.jsonl'):
      path_by_type['queries'] = f
    elif f.endswith('corpus.jsonl'):
      path_by_type['corpus'] = f
    elif f.endswith('train.tsv'):
      path_by_type['train'] = f
    elif f.endswith('test.tsv'):
      path_by_type['test'] = f

  return path_by_type


def download_from_gcs(blob_uri: str, redownload: bool = False) -> str:
  blob_path = blob_uri[5:]
  output_path = os.path.join(TEMP_DIR, blob_uri)
  if redownload or (not os.path.isfile(output_path)):
    output_dir = os.path.dirname(output_path)
    os.makedirs(output_dir, exist_ok=True)
    with open(output_path, 'wb') as f:
      STORAGE_CLIENT.download_blob_to_file(blob_uri, file_obj=f)

  return output_path


def load_jsonl_from_gcs(
    blob_uri: str, nrows: Optional[int] = None, redownload: bool = False
) -> pd.DataFrame:
  output_path = download_from_gcs(blob_uri, redownload=redownload)
  return pd.read_json(output_path, lines=True, nrows=nrows, dtype={'_id': str})


def load_tsv_from_gcs(
    blob_uri: str, nrows: Optional[int] = None, redownload: bool = False
) -> pd.DataFrame:
  output_path = download_from_gcs(blob_uri, redownload=redownload)
  return pd.read_csv(output_path, sep='\t', dtype={'_id': str})

# Download the CQADupStack English dataset and export it to GCS

In [15]:
# Download the data to GCS and get the paths
output_paths = download_to_gcs(DATA_URL, BUCKET_NAME, INPUT_DATA_PREFIX)
path_by_type = get_path_by_type(output_paths)

OUTPUT_DIR = os.path.join(
    os.path.dirname(path_by_type['queries']),
    OUTPUT_SUBFOLDER,
    )
path_by_type

gs://my-project-0004-bucket/download/beir/cqadupstack/english/corpus.jsonl does not exist


Finished downloading https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/cqadupstack.zip to /var/tmp/tmpqu4siffx/b2d1f3c65bc8aa4cd7b2358d69fcb8df98fcc4f2486e877c.zip
Extracting cqadupstack/english/ from /var/tmp/tmpqu4siffx/b2d1f3c65bc8aa4cd7b2358d69fcb8df98fcc4f2486e877c.zip
Extracting cqadupstack/english/qrels/ from /var/tmp/tmpqu4siffx/b2d1f3c65bc8aa4cd7b2358d69fcb8df98fcc4f2486e877c.zip
Extracting cqadupstack/english/qrels/test.tsv from /var/tmp/tmpqu4siffx/b2d1f3c65bc8aa4cd7b2358d69fcb8df98fcc4f2486e877c.zip
Extracting cqadupstack/english/corpus.jsonl from /var/tmp/tmpqu4siffx/b2d1f3c65bc8aa4cd7b2358d69fcb8df98fcc4f2486e877c.zip
Extracting cqadupstack/english/queries.jsonl from /var/tmp/tmpqu4siffx/b2d1f3c65bc8aa4cd7b2358d69fcb8df98fcc4f2486e877c.zip
Uploading /var/tmp/tmpx14cv328/cqadupstack/english/qrels/train.tsv to gs://my-project-0004-bucket/download/beir/cqadupstack/english/qrels/train.tsv
Uploading /var/tmp/tmpx14cv328/cqadupstack/english/qrels/test.tsv to gs

{'train': 'gs://my-project-0004-bucket/download/beir/cqadupstack/english/qrels/train.tsv',
 'test': 'gs://my-project-0004-bucket/download/beir/cqadupstack/english/qrels/test.tsv',
 'queries': 'gs://my-project-0004-bucket/download/beir/cqadupstack/english/queries.jsonl',
 'corpus': 'gs://my-project-0004-bucket/download/beir/cqadupstack/english/corpus.jsonl'}

# Show some example data

In [16]:
#@markdown Example Queries. { display-mode: "form" }
number_of_example_queries = 10 # @param {type:"integer"}
query_df = load_jsonl_from_gcs(path_by_type['queries'], nrows=number_of_example_queries).set_index('_id')
query_df[['text']].head(number_of_example_queries)

,text
_id,
19399,"Is ""a wide range of features"" singular or plural?"
5987,Is there any rule for the placement of space after and before parenthesis?
57488,Word for two people who are the same age
97154,What’s the pronunciation of “ s’ ”?
270,What is the correct plural of octopus?
195514,"is ""is"" or ""are correct?"
88027,What weather! What a pity! - phrases with and without article - why?
53730,"""Aren't I"" vs ""Amn't I"""
10705,"When do the ""-uple""s end?"


In [17]:
#@markdown Example Documents. { display-mode: "form" }
number_of_example_documents = 10 # @param {type:"integer"}
corpus_df = load_jsonl_from_gcs(path_by_type['corpus'], nrows=number_of_example_queries).set_index('_id')
corpus_df[['text']].head(number_of_example_documents)

,text
_id,
11547,An eponym is one way to eternal (if posthumous) fame. But is there a word meaning an eponym some...
11549,"We are working on a web project that has a password reset feature. Now the problem is, between ""..."
123851,"There are many homonyms in the English language, words that are spelled the same and pronounced ..."
116661,Some say the following two phrases are equivalent because of Raising (linguistics)! Example 1 > ...
102236,"Please tell me if the following sentence requires ""have"" or ""has"": > My degree in Cell Biology a..."
91901,> **Possible Duplicate:** > Is the usage of “are” correct when referring to a team/group/band...
177507,Let's say we have a group of people who starts an activity together. The name of group can be: ...
80798,"> **Possible Duplicate:** > Are collective nouns always plural, or are certain ones singular?..."
112990,> 1) Our team of nationally recognized trainers has earned multiple titles…. In the first versio...


In [18]:
train_df = load_tsv_from_gcs(path_by_type['train'])
train_df.head()

,query-id,corpus-id,score
0,100173,147988,1
1,100414,60956,1
2,100605,184738,1
3,100664,8892,1
4,100664,25375,1
